In [1]:
import qiskit_nature
import qiskit
import pyscf

print("Qiskit version:", qiskit.__version__)
print("Qiskit Nature version:", qiskit_nature.__version__)
print("PySCF version:", pyscf.__version__)


Qiskit version: 1.4.2
Qiskit Nature version: 0.7.2
PySCF version: 2.8.0


In [2]:
#H2 CCSD energy


from pyscf import gto, scf, cc

mol = gto.M(
    atom='H 0 0 0; H 0 0 0.735',
    basis='sto-3g',
    unit='Angstrom',
    charge=0,
    spin=0,
)

mf = scf.RHF(mol).run()
mycc = cc.CCSD(mf).run()

print("H2 PySCF CCSD Energy:", mycc.e_tot)


converged SCF energy = -1.116998996754
E(CCSD) = -1.137306193391969  E_corr = -0.02030719663796497
H2 PySCF CCSD Energy: -1.1373061933919688


In [ ]:
#H2 VQE(no freezecore+no tapering)
#uccsd ansatz

from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.transformers import FreezeCoreTransformer
from qiskit.quantum_info import Z2Symmetries
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD
from qiskit_aer.primitives import EstimatorV2
from qiskit.providers.basic_provider import BasicSimulator
from scipy.optimize import minimize
import numpy as np

# Set up the molecule H2
molecule = MoleculeInfo(["H", "H"], [(0.0, 0.0, 0.0), (0.0, 0.0, 0.735)], charge=0, multiplicity=1)
driver = PySCFDriver.from_molecule(molecule, basis="sto3g")
problem = driver.run()


hamiltonian = problem.hamiltonian.second_q_op()
mapper = JordanWignerMapper()

qubit_op = mapper.map(hamiltonian)

backend = BasicSimulator()

num_particles = problem.num_particles
num_spatial_orbitals = problem.num_spatial_orbitals
hf_state = HartreeFock(num_spatial_orbitals, num_particles, mapper)
ansatz = UCCSD(num_spatial_orbitals, num_particles, mapper, initial_state=hf_state)



print(f"Circuit has {ansatz.num_parameters} parameters")
print(f"Parameter names: {ansatz.parameters}")

initial_params = np.zeros(ansatz.num_parameters)
# initial_params = np.random.uniform(-0.1, 0.1, ansatz.num_parameters)

estimator = EstimatorV2()
cost_history = []

def cost_func(params):
    bound_circ = ansatz.assign_parameters(params)
    pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
    transpiled_ansatz = pm.run(bound_circ)
    pub = (transpiled_ansatz, [qubit_op], [np.array([])])
    result = estimator.run([pub]).result()
    energy = result[0].data.evs[0]
    cost_history.append(energy)
    print(f"  Iteration {len(cost_history)} -> Energy: {energy:.6f}")
    return energy

# Run the VQE optimization
print("\nRunning VQE optimization for H2 without freeze core and qubit tapering...")
result = minimize(cost_func, initial_params, method="COBYLA")
nuclearenergy=problem.nuclear_repulsion_energy  
# Print final results
print("\nOptimization complete!")
print(f"Value from VQE: {result.fun :.6f} Hartree")
print(f"Nuclear energy: { nuclearenergy:.6f} Hartree")
print(f"Final ground state energy of H2: {result.fun + nuclearenergy:.6f} Hartree")
print(f"Optimization success: {result.success}")
print(f"Number of iterations: {len(cost_history)}")



Circuit has 3 parameters
Parameter names: ParameterView([ParameterVectorElement(t[0]), ParameterVectorElement(t[1]), ParameterVectorElement(t[2])])

Running VQE optimization for H2 with freeze core and qubit tapering...
  Iteration 1 -> Energy: -1.836968
  Iteration 2 -> Energy: -1.289404
  Iteration 3 -> Energy: -1.289404
  Iteration 4 -> Energy: -0.545372
  Iteration 5 -> Energy: -1.234006
  Iteration 6 -> Energy: -1.676179
  Iteration 7 -> Energy: -1.812798
  Iteration 8 -> Energy: -1.840429
  Iteration 9 -> Energy: -1.692616
  Iteration 10 -> Energy: -1.848329
  Iteration 11 -> Energy: -1.844031
  Iteration 12 -> Energy: -1.813097
  Iteration 13 -> Energy: -1.854058
  Iteration 14 -> Energy: -1.853218
  Iteration 15 -> Energy: -1.855430
  Iteration 16 -> Energy: -1.856186
  Iteration 17 -> Energy: -1.856221
  Iteration 18 -> Energy: -1.856751
  Iteration 19 -> Energy: -1.856521
  Iteration 20 -> Energy: -1.856498
  Iteration 21 -> Energy: -1.857209
  Iteration 22 -> Energy: -1.8569

In [6]:
#H2 VQE + freeze core + qubit tapering 
#UCCSD ansatz
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.transformers import FreezeCoreTransformer
from qiskit.quantum_info import Z2Symmetries
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD
from qiskit_aer.primitives import EstimatorV2
from qiskit.providers.basic_provider import BasicSimulator
from scipy.optimize import minimize
import numpy as np
from qiskit.circuit.library import EfficientSU2

molecule = MoleculeInfo(["H", "H"], [(0.0, 0.0, 0.0), (0.0, 0.0, 0.735)], charge=0, multiplicity=1)
driver = PySCFDriver.from_molecule(molecule, basis="sto3g")
problem = driver.run()

print("Applying freeze core approximation...")
fc_transformer = FreezeCoreTransformer()
transformed_problem = fc_transformer.transform(problem)

hamiltonian = transformed_problem.second_q_ops()[0]
mapper = JordanWignerMapper()

tapered_mapper = transformed_problem.get_tapered_mapper(mapper)
qubit_op = tapered_mapper.map(hamiltonian)


backend = BasicSimulator()

num_particles = transformed_problem.num_particles
num_spatial_orbitals = transformed_problem.num_spatial_orbitals
hf_state = HartreeFock(num_spatial_orbitals, num_particles, tapered_mapper)
ansatz = UCCSD(num_spatial_orbitals, num_particles, tapered_mapper, initial_state=hf_state)



print(f"Circuit has {ansatz.num_parameters} parameters")
print(f"Parameter names: {ansatz.parameters}")

initial_params = np.zeros(ansatz.num_parameters)
# initial_params = np.random.uniform(-0.1, 0.1, ansatz.num_parameters)

estimator = EstimatorV2()
cost_history = []

def cost_func(params):
    bound_circ = ansatz.assign_parameters(params)
    pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
    transpiled_ansatz = pm.run(bound_circ)
    pub = (transpiled_ansatz, [qubit_op], [np.array([])])
    result = estimator.run([pub]).result()
    energy = result[0].data.evs[0]
    cost_history.append(energy)
    print(f"  Iteration {len(cost_history)} -> Energy: {energy:.6f}")
    return energy

print("\nRunning VQE optimization for H2 with freeze core and qubit tapering...")
result = minimize(cost_func, initial_params, method="COBYLA")
nuclearenergy=problem.nuclear_repulsion_energy  
frozen_core_energy = transformed_problem.hamiltonian.constants.get("FreezeCoreTransformer", 0.0)
# Print final results
print("\nOptimization complete!")
print(f"Value from VQE: {result.fun :.6f} Hartree")
print(f"Nuclear energy: { nuclearenergy:.6f} Hartree")
print(f"Frozen core energy from hamiltonian constants: {frozen_core_energy} Hartree")
print(f"Final ground state energy of H2: {result.fun + nuclearenergy:.6f} Hartree")

print(f"Optimization success: {result.success}")
print(f"Number of iterations: {len(cost_history)}")

Applying freeze core approximation...
Circuit has 1 parameters
Parameter names: ParameterView([ParameterVectorElement(t[0])])

Running VQE optimization for H2 with freeze core and qubit tapering...
  Iteration 1 -> Energy: -1.836968
  Iteration 2 -> Energy: -0.545372
  Iteration 3 -> Energy: -0.874413
  Iteration 4 -> Energy: -1.623355
  Iteration 5 -> Energy: -1.652796
  Iteration 6 -> Energy: -1.856989
  Iteration 7 -> Energy: -1.826282
  Iteration 8 -> Energy: -1.847931
  Iteration 9 -> Energy: -1.856745
  Iteration 10 -> Energy: -1.855916
  Iteration 11 -> Energy: -1.857227
  Iteration 12 -> Energy: -1.857266
  Iteration 13 -> Energy: -1.857105
  Iteration 14 -> Energy: -1.857210
  Iteration 15 -> Energy: -1.857275
  Iteration 16 -> Energy: -1.857271
  Iteration 17 -> Energy: -1.857275
  Iteration 18 -> Energy: -1.857274
  Iteration 19 -> Energy: -1.857275
  Iteration 20 -> Energy: -1.857275
  Iteration 21 -> Energy: -1.857275
  Iteration 22 -> Energy: -1.857275
  Iteration 23 -> E

In [7]:
#LiH CCSD energy

mol = gto.M(
    atom='Li 0 0 0; H 0 0 1.6',
    basis='sto-3g',
    unit='Angstrom',
    charge=0,
    spin=0,
)

mf = scf.RHF(mol).run()
mycc = cc.CCSD(mf).run()

print("LiH PySCF CCSD Energy (no freeze core):", mycc.e_tot)


converged SCF energy = -7.86186476980865
E(CCSD) = -7.882313821664069  E_corr = -0.02044905185541488
LiH PySCF CCSD Energy (no freeze core): -7.882313821664069


In [ ]:
# LiH CCSD energy with frozen core

mol = gto.M(
    atom='Li 0 0 0; H 0 0 1.6',
    basis='sto-3g',
    unit='Angstrom',
    charge=0,
    spin=0,
)

mf = scf.RHF(mol).run()
mycc = cc.CCSD(mf, frozen=1).run()  
print("LiH PySCF CCSD Energy (with freeze core):", mycc.e_tot)


converged SCF energy = -7.86186476980865
E(CCSD) = -7.882096600731343  E_corr = -0.02023183092268855
LiH PySCF CCSD Energy (with freeze core): -7.882096600731343


In [ ]:
#LiH VQE + Freezecore + QubitTapering

from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.transformers import FreezeCoreTransformer
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD
from qiskit_aer.primitives import EstimatorV2
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit.providers.basic_provider import BasicSimulator
from scipy.optimize import minimize
import numpy as np

molecule = MoleculeInfo(["Li", "H"], [(0.0, 0.0, 0.0), (0.0, 0.0, 1.6)], charge=0, multiplicity=1)
driver = PySCFDriver.from_molecule(molecule, basis="sto3g")
problem = driver.run()

fc_transformer = FreezeCoreTransformer()
transformed_problem = fc_transformer.transform(problem)


hamiltonian = transformed_problem.second_q_ops()[0]

mapper = JordanWignerMapper()

tapered_mapper = transformed_problem.get_tapered_mapper(mapper)
qubit_op = tapered_mapper.map(hamiltonian)

print(f"Number of qubits after tapering: {qubit_op.num_qubits}")

backend = BasicSimulator()

num_particles = transformed_problem.num_particles
num_spatial_orbitals = transformed_problem.num_spatial_orbitals

hf_state = HartreeFock(num_spatial_orbitals, num_particles, tapered_mapper)
ansatz = UCCSD(num_spatial_orbitals, num_particles, tapered_mapper, initial_state=hf_state)

print(f"Circuit has {ansatz.num_parameters} parameters")

initial_params = np.random.uniform(-0.1, 0.1, ansatz.num_parameters)

estimator = EstimatorV2()
cost_history = []

def cost_func(params):
    bound_circ = ansatz.assign_parameters(params)
    pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
    transpiled_ansatz = pm.run(bound_circ)
    pub = (transpiled_ansatz, [qubit_op], [np.array([])])
    result = estimator.run([pub]).result()
    energy = result[0].data.evs[0]
    cost_history.append(energy)
    print(f"  Iteration {len(cost_history)} -> Energy: {energy:.6f}")
    return energy

print("\nRunning VQE optimization for LiH with freeze core and qubit tapering...")
result = minimize(cost_func, initial_params, method="COBYLA", options={'maxiter': 200})

# Print final results
print("\nOptimization complete!")
vqe_energy = result.fun
print(f"VQE energy (active space only): {vqe_energy:.6f} Hartree")

nuclearenergy = problem.nuclear_repulsion_energy
frozen_core_energy = transformed_problem.hamiltonian.constants.get("FreezeCoreTransformer", 0.0)
total_energy = vqe_energy + frozen_core_energy + nuclearenergy
print(f"Nuclear repulsion energy: {nuclearenergy} Hartree")
print(f"Frozen core energy from hamiltonian constants: {frozen_core_energy} Hartree")
print(f"Total ground state energy of LiH: {total_energy:.6f} Hartree")
print(f"Expected CCSD energy: -7.882097 Hartree")
print(f"Difference from expected: {total_energy - (-7.882097):.6f} Hartree")
print(f"Optimization success: {result.success}")
print(f"Number of iterations: {len(cost_history)}")

Frozen core energy from hamiltonian constants: -7.796219568777053 Hartree
Number of qubits after tapering: 6
Circuit has 10 parameters

Running VQE optimization for LiH with freeze core and qubit tapering...
  Iteration 1 -> Energy: -0.997704
  Iteration 2 -> Energy: -0.947779
  Iteration 3 -> Energy: -0.743335
  Iteration 4 -> Energy: -0.942739
  Iteration 5 -> Energy: -0.736205
  Iteration 6 -> Energy: -0.512902
  Iteration 7 -> Energy: -0.608755
  Iteration 8 -> Energy: -0.576162
  Iteration 9 -> Energy: -0.503437
  Iteration 10 -> Energy: -0.588547
  Iteration 11 -> Energy: -0.143836
  Iteration 12 -> Energy: -0.621718
  Iteration 13 -> Energy: -0.917857
  Iteration 14 -> Energy: -0.958115
  Iteration 15 -> Energy: -0.997433
  Iteration 16 -> Energy: -1.011374
  Iteration 17 -> Energy: -0.996358
  Iteration 18 -> Energy: -0.890720
  Iteration 19 -> Energy: -0.991978
  Iteration 20 -> Energy: -1.007033
  Iteration 21 -> Energy: -1.013405
  Iteration 22 -> Energy: -1.024703
  Iterati